In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm_notebook
np.set_printoptions(suppress=True)
from sklearn import preprocessing
from tqdm import tqdm
import decimal
from itertools import combinations
from random import shuffle
from time import sleep
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
import gc
import datetime
from sklearn.model_selection import train_test_split, KFold, GroupKFold
import os
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
import random
import lightgbm as lgb

In [2]:
#Always seed the randomness of this universe
def seed_everything(seed=51):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
seed_everything(seed=51)

In [4]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [5]:
%%time
train = pd.read_csv('../input/train4.csv')
print("train finished")

train finished
CPU times: user 50.3 s, sys: 4.13 s, total: 54.5 s
Wall time: 54.5 s


In [6]:
%%time
test = pd.read_csv('../input/test4.csv')
print("test finished")

test finished
CPU times: user 42.2 s, sys: 1.95 s, total: 44.1 s
Wall time: 44.1 s


In [7]:
#sample_submission
sample_submission = pd.read_csv('../input/sample_submission.csv')
print(train.shape,test.shape,sample_submission.shape)

(590540, 915) (506691, 914) (506691, 2)


In [8]:
#y = train['isFraud']
#del train['isFraud']
gc.collect()
print(train.shape,test.shape)

(590540, 915) (506691, 914)


In [9]:
"""importances = pd.read_csv('./importances/importances1.csv')
print(importances.shape)
importances = importances[:700]
features = [x for x in list(importances['index']) if x not in ['TransactionID','TransactionDT']]
train = train[features]
test = test[features]
"""

"importances = pd.read_csv('./importances/importances1.csv')\nprint(importances.shape)\nimportances = importances[:700]\nfeatures = [x for x in list(importances['index']) if x not in ['TransactionID','TransactionDT']]\ntrain = train[features]\ntest = test[features]\n"

In [10]:
gc.collect()

0

In [ ]:
print(train.shape,test.shape)

In [ ]:
train.head()

In [ ]:
cat_cols = ['ProductCD','card1','card2','card3','card4','card5','dist1','dist2','P_emaildomain','R_emaildomain','DeviceInfo']
print(len(cat_cols))

In [ ]:
train[cat_cols].nunique()

In [ ]:
def convert_less_common(x):
    if x not in shared_list:
        return np.nan
    else:
        return x

In [ ]:
for tuple_ in list(combinations(cat_cols,2)):
    f1 = tuple_[0]
    f2 = tuple_[1]
    column_name = f1 + '_' + f2
    
    train[column_name] = train[f1].apply(str) + '_' + train[f2].apply(str)
    test[column_name] = test[f1].apply(str) +  '_' +test[f2].apply(str)
    
    shared_list = list( set(train[column_name]).intersection(set(test[column_name] )) )
    shared = len(shared_list)
    
    only_train = train[column_name].nunique() - len(shared_list)
    only_test = test[column_name].nunique() - len(shared_list)
    print(column_name)
    train[column_name] = train[column_name].apply(convert_less_common)
    test[column_name] = test[column_name].apply(convert_less_common)
    
    print( column_name,round(shared/(shared + only_train),3) )
    print( column_name,round(shared/(shared + only_test),3) )
    print("="*100)
    
    del shared_list,shared,only_train,only_test
    gc.collect()

In [ ]:
for tuple_ in tqdm(list(combinations(cat_cols,2))):
    f1 = tuple_[0]
    f2 = tuple_[1]
    column_name = f1 + '_' + f2
    train[column_name + '_isnull'] = train[column_name].isnull()*1
    test[column_name + '_isnull'] = test[column_name].isnull()*1    

In [ ]:
list(combinations(cat_cols,2))[:5]

In [ ]:
print(train.shape,test.shape)

In [ ]:
for tuple_ in tqdm(list(combinations(cat_cols,2))):
    f1 = tuple_[0]
    f2 = tuple_[1]
    column_name = f1 + '_' + f2
    train[column_name] = train[column_name].fillna('999999')
    test[column_name] = test[column_name].fillna('999999')
    le = LabelEncoder()
    le.fit(list(train[column_name]) + list(test[column_name]))
    train[column_name] = le.transform(train[column_name])
    test[column_name] = le.transform(test[column_name])
print(train.shape,test.shape)

In [ ]:
train[column_name].head()

In [ ]:
%%time
train.to_csv('../input/train5.csv',index=False)
test.to_csv('../input/test5.csv',index=False)

# Training

In [17]:
y = train['isFraud']
del train['isFraud']

In [18]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.02,
                    'num_leaves': 2**8,
                    'max_depth':12,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.5,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':2000,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 51,
                    'early_stopping_rounds':100, 
                } 

In [ ]:
gc.collect()

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
%%time
train = reduce_mem_usage(train)

In [ ]:
%%time
test = reduce_mem_usage(test)

In [ ]:
gc.collect()

In [19]:
N = 5
kf = KFold(n_splits=N)

importance = pd.DataFrame(np.zeros((train.shape[1], N)), columns=['Fold_{}'.format(i) for i in range(1, N + 1)], index=train.columns)
scores = []
y_pred = np.zeros(test.shape[0])
oof = np.zeros(train.shape[0])

for fold, (trn_idx, val_idx) in enumerate(kf.split(train, y), 1):
    print('Fold {}'.format(fold))
          
    trn_data = lgb.Dataset(train.iloc[trn_idx, :].values, label=y.iloc[trn_idx].values)
    val_data = lgb.Dataset(train.iloc[val_idx, :].values, label=y.iloc[val_idx].values)   
    
    clf = lgb.train(lgb_params, trn_data ,valid_sets=[trn_data, val_data], verbose_eval=100)

    predictions = clf.predict(train.iloc[val_idx, :].values) 
    importance.iloc[:, fold - 1] = clf.feature_importance()
    oof[val_idx] = predictions

    score = roc_auc_score(y.iloc[val_idx].values, predictions)
    scores.append(score)
    print('Fold {} ROC AUC Score {}\n'.format(fold, score))

    y_pred += clf.predict(test) / N
    
    del trn_data, val_data, predictions
    gc.collect()
    
print('Average ROC AUC Score {} [STD:{}]'.format(np.mean(scores), np.std(scores)))

Fold 1


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.956543	valid_1's auc: 0.891993
[200]	training's auc: 0.980104	valid_1's auc: 0.907963
[300]	training's auc: 0.9881	valid_1's auc: 0.915931
[400]	training's auc: 0.992258	valid_1's auc: 0.920552
[500]	training's auc: 0.994866	valid_1's auc: 0.923222
[600]	training's auc: 0.99664	valid_1's auc: 0.924537
[700]	training's auc: 0.997723	valid_1's auc: 0.925969
[800]	training's auc: 0.998508	valid_1's auc: 0.926367
[900]	training's auc: 0.99906	valid_1's auc: 0.926812
[1000]	training's auc: 0.999438	valid_1's auc: 0.927076
[1100]	training's auc: 0.999654	valid_1's auc: 0.926817
Early stopping, best iteration is:
[1007]	training's auc: 0.999446	valid_1's auc: 0.927162
Fold 1 ROC AUC Score 0.9271622042838722

Fold 2


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.954117	valid_1's auc: 0.910675
[200]	training's auc: 0.97988	valid_1's auc: 0.92624
[300]	training's auc: 0.989143	valid_1's auc: 0.932175
[400]	training's auc: 0.993131	valid_1's auc: 0.9349
[500]	training's auc: 0.995543	valid_1's auc: 0.937144
[600]	training's auc: 0.997114	valid_1's auc: 0.938512
[700]	training's auc: 0.998136	valid_1's auc: 0.939271
[800]	training's auc: 0.998819	valid_1's auc: 0.940008
[900]	training's auc: 0.999296	valid_1's auc: 0.940718
[1000]	training's auc: 0.999599	valid_1's auc: 0.941202
[1100]	training's auc: 0.999779	valid_1's auc: 0.941699
[1200]	training's auc: 0.99988	valid_1's auc: 0.941678
Early stopping, best iteration is:
[1122]	training's auc: 0.999815	valid_1's auc: 0.941847
Fold 2 ROC AUC Score 0.9418471644497299

Fold 3


...

Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.9572	valid_1's auc: 0.911297
[200]	training's auc: 0.981659	valid_1's auc: 0.925559
[300]	training's auc: 0.990125	valid_1's auc: 0.930725
[400]	training's auc: 0.993945	valid_1's auc: 0.93244
[500]	training's auc: 0.995915	valid_1's auc: 0.933198
[600]	training's auc: 0.997255	valid_1's auc: 0.933151
[700]	training's auc: 0.998221	valid_1's auc: 0.933681
[800]	training's auc: 0.998966	valid_1's auc: 0.933867
[900]	training's auc: 0.999373	valid_1's auc: 0.933848
Early stopping, best iteration is:
[886]	training's auc: 0.999333	valid_1's auc: 0.933995
Fold 3 ROC AUC Score 0.9339950429070237

Fold 4


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.953622	valid_1's auc: 0.925665
[200]	training's auc: 0.980517	valid_1's auc: 0.94139
[300]	training's auc: 0.989093	valid_1's auc: 0.947232
[400]	training's auc: 0.993245	valid_1's auc: 0.949728
[500]	training's auc: 0.995704	valid_1's auc: 0.95097
[600]	training's auc: 0.997336	valid_1's auc: 0.951356
[700]	training's auc: 0.998304	valid_1's auc: 0.95142
Early stopping, best iteration is:
[683]	training's auc: 0.998188	valid_1's auc: 0.951504
Fold 4 ROC AUC Score 0.9515041433324347

Fold 5


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.954619	valid_1's auc: 0.902239
[200]	training's auc: 0.980536	valid_1's auc: 0.919845
[300]	training's auc: 0.989476	valid_1's auc: 0.925148
[400]	training's auc: 0.993589	valid_1's auc: 0.928
[500]	training's auc: 0.995881	valid_1's auc: 0.929364
[600]	training's auc: 0.997255	valid_1's auc: 0.929772
[700]	training's auc: 0.998254	valid_1's auc: 0.929689
Early stopping, best iteration is:
[606]	training's auc: 0.997338	valid_1's auc: 0.929891
Fold 5 ROC AUC Score 0.9298906329357063

Average ROC AUC Score 0.9368798375817533 [STD:0.008833437311879187]


In [20]:
print(roc_auc_score(y,oof))

0.9354840326091465


In [21]:
importance['avg'] = importance.mean(axis=1)

In [22]:
importance = importance.sort_values(by='avg',ascending=False)

In [23]:
importance = importance.reset_index(drop=False)

In [24]:
importance.head()

,index,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,avg
0,TransactionID,6898,7619,6205,4471,3755,5789.6
1,TransactionAmt,6430,7404,5594,4257,3924,5521.8
2,addr1,4686,5689,4052,3092,2773,4058.4
3,card1,4798,5197,4259,3056,2714,4004.8
4,day,3846,4515,3256,2442,2042,3220.2


In [25]:
importance.to_csv('../importances/importances7.csv',index=False)

In [26]:
sample_submission.head()

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5


In [27]:
sample_submission['isFraud'] = y_pred

In [28]:
sample_submission.head()

,TransactionID,isFraud
0,3663549,0.001333
1,3663550,0.001398
2,3663551,0.002326
3,3663552,0.001364
4,3663553,0.001672


In [29]:
sample_submission.to_csv('../predictions/pred7.csv',index=False)

In [30]:
!kaggle competitions submit -c ieee-fraud-detection -f ../predictions/pred7.csv -m "pred7"

100%|███████████████████████████████████████| 14.1M/14.1M [00:33<00:00, 444kB/s]
Successfully submitted to IEEE-CIS Fraud Detection